## ⚙️ 1. Setup: Install Libraries

First, let's install the specific version of the Google Agent Development Kit (ADK) that this notebook is built with. Pinning the version ensures our code will always work as expected.

In [ ]:
!pip install google-adk==1.8.0 -q

## 🔑 2. Authentication: Configure Your API Key

Next, we need to securely provide our Google API key. This code will create a secure input prompt for you to paste your key. It then sets the key as an environment variable, which is the standard way the ADK authenticates your requests.

In [ ]:
import os
from getpass import getpass

# Prompt the user for their API key securely
api_key = getpass('Enter your Google API Key: ')

# Set the API key as an environment variable for ADK to use
os.environ['GOOGLE_API_KEY'] = api_key

print("✅ API Key configured successfully! Let the fun begin.")

## 🛠️ 3. Define Custom Tools

Here, we define the custom skills for our guest management agent. These are simple Python functions that interact with a dictionary acting as our guest database. The ADK will use the function names, signatures, and docstrings to understand how and when to call them.

In [ ]:
import json

# Simple dictionary to act as our guest list database.
GUEST_DATABASE = {}

def add_guest(name: str, email: str) -> str:
  """Adds a guest's name and email to the event's guest list."""
  print(f"Tool executed: Adding guest '{name}' with email '{email}'.")
  GUEST_DATABASE[email] = name
  return f"Successfully added {name} to the guest list."

def get_guest_list() -> str:
  """Retrieves the current list of all registered guests for the event."""
  print("Tool executed: Retrieving guest list.")
  if not GUEST_DATABASE:
    return "The guest list is currently empty."
  return json.dumps(GUEST_DATABASE)

## 🧑‍💼 4. Create Specialist Agent 1: The Guest Manager

This is our first specialist agent. Its only job is to manage the event's guest list. We provide it with the add_guest and get_guest_list functions as its tools and give it clear instructions on how to use them.

In [ ]:
from google.adk.agents import Agent

guest_management_agent = Agent(
    name="guest_management_agent",
    model="gemini-2.5-flash",
    description="A specialized agent for managing an event's guest list. It can add guests and retrieve the current list.",
    instruction="""
    You are a guest management assistant. Your only job is to add guests to a list
    or retrieve the full list when asked.

    - Use the `add_guest` tool to add a new person to the list.
    - Use the `get_guest_list` tool to show the current guest list.

    You do not perform any other tasks like planning events or searching for information.
    Stick strictly to managing the guest list with your tools.
    """,
    tools=[add_guest, get_guest_list]
)

## 🧑‍🎨 5. Create Specialist Agent 2: The Event Planner

This is our second specialist. Its expertise lies in planning events using the built-in Google Search tool. It is instructed to handle all aspects of planning but knows nothing about managing guests.

In [ ]:
from google.adk.tools import google_search

def create_event_planner_agent():
    """Create event planner agent"""
    return Agent(
        name="event_planner_agent",
        model="gemini-2.5-flash",
        description="A specialist agent that plans events by searching for themes, venues, and schedules. It can revise the plan based on user feedback.",
        instruction="""
        You are an expert event planner. Your only goal is to create a comprehensive plan for an event.

        Your process is as follows:
        1.  **Brainstorm Themes:** Suggest 2-3 creative themes.
        2.  **Find Venues:** Use your search tool to find 3-5 potential venues.
        3.  **Create an Agenda:** Propose a high-level schedule for the event.
        4.  **Summarize Clearly:** Present the final plan in an organized format.

        You do not handle guest lists or any other tasks. Stick to planning.

        Always be friendly and confirm your actions to the user.
        """,
        tools=[google_search]
    )

event_planner_agent = create_event_planner_agent()
print(f"🧞 Agent '{event_planner_agent.name}' is created and ready!")

## 👔 6. Assemble the Agent Team

This is the core of the "Agent-as-a-Tool" pattern. We use AgentTool to wrap our two specialist agents, making them callable as tools. We then create a "manager" agent, the root_orchestrator_agent, whose only job is to delegate tasks to its team members.

In [ ]:
from google.adk.tools.agent_tool import AgentTool

event_planner_tool = AgentTool(agent=event_planner_agent)
guest_list_manager_tool = AgentTool(agent=guest_management_agent)

In [ ]:
root_orchestrator_agent = Agent(
    name="root_orchestrator_agent",
    model="gemini-2.5-flash",
    description="The main orchestrator that delegates tasks to specialist agents for event planning and guest management.",
    instruction="""
    You are the root orchestrator managing an ongoing conversation. Your job is to understand the full context of the user's request and delegate it to the correct specialist tool.

    - If the user wants to plan an event, find venues, or create a schedule, use the `event_planner_tool`.
    - If the user wants to add, view, or manage guests, use the `guest_list_manager_tool`.

    Do not try to answer the user's query directly. Always delegate to the appropriate tool. Do not summarize responses from the tools.

    """,
    tools=[event_planner_tool, guest_list_manager_tool]
)

## 🚀 7. Build the Execution Engine

This is our helper function to run queries. It handles the core ADK logic: initializing the Runner, streaming events with run_async, and displaying the final response. We'll use this function to interact with our orchestrator.

In [ ]:
from IPython.display import display, Markdown

from google.genai.types import Content, Part
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session


async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    print("\n" + "-"*50)
    print("✅ Final Response:")
    display(Markdown(final_response))
    print("-"*50 + "\n")

    return final_response

## 🧠 8. Initialize Session Service and Run

Finally, we set up our InMemorySessionService and define our main execution block. This code will create a single session for the orchestrator and then send it a series of queries to demonstrate how it intelligently routes tasks to the correct specialist agent.

In [ ]:
# --- Initialize our Session Service ---
# This one service will manage all the different sessions in our notebook.
session_service = InMemorySessionService()
user_id = "adk_event_planner_001"

In [ ]:
async def run_stateful_orchestrator():

  session = await session_service.create_session(
        app_name=root_orchestrator_agent.name,
        user_id=user_id
  )

  query1 = "Plan an AI/ML meetup in New York"
  print(f"User: {query1}\n")
  await run_agent_query(root_orchestrator_agent, query1, session, user_id)

  query2 = "Add 'Tim Apple' with email 'tim.a@example.com' to the guest list."
  print(f"User: {query2}\n")
  await run_agent_query(root_orchestrator_agent, query2, session, user_id)

  query3 = "Now show me the guest list."
  await run_agent_query(root_orchestrator_agent, query3, session, user_id)
# Run the full system
await run_stateful_orchestrator()